In [1]:
import tensorflow as tf
import datetime
import os
import numpy as np
import ctypes
from PIL import Image

In [2]:
#### ------------------------------------------- ####
#### ------------- LOAD THE DATA --------------- ####
#### ------------------------------------------- ####

In [3]:
MAIN_DATASET_FOLDER = "..\\dataset\\"

TEST_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "test")
TRAIN_SUBFOLDER = os.path.join(MAIN_DATASET_FOLDER, "train")

TEST_ESPAGNE_FOLDER = os.path.join(TEST_SUBFOLDER, "espagne")
TEST_FRANCE_FOLDER = os.path.join(TEST_SUBFOLDER, "france")
TEST_JAPON_FOLDER = os.path.join(TEST_SUBFOLDER, "japon")

TRAIN_ESPAGNE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "espagne")
TRAIN_FRANCE_FOLDER = os.path.join(TRAIN_SUBFOLDER, "france")
TRAIN_JAPON_FOLDER = os.path.join(TRAIN_SUBFOLDER, "japon")


In [4]:
def fill_x_and_y_with_images_and_labels(folder, classe, dataset):

    for file in os.listdir(folder):

        file_path = os.path.join(folder, file)
        image = Image.open(file_path)
        image = image.resize((64, 64))
        if image.mode in ("RGB", "P"): image = image.convert("RGBA")
        im_arr = np.array(image).flatten()
        im_arr = im_arr / 255.0

        completed_dataset = {}
        completed_dataset["value"] = im_arr
        completed_dataset["classe"] = classe

        dataset.append(completed_dataset.copy())

In [5]:
def import_dataset(folder, classe):

    dataset = []

    fill_x_and_y_with_images_and_labels(folder, classe, dataset)

    return dataset

In [6]:
x_test = []
y_test = []

x_train = []
y_train = []

class_espagne_expected_outputs = [1, 0, 0]
dataset_test_for_espagne = import_dataset(TEST_ESPAGNE_FOLDER, class_espagne_expected_outputs)
dataset_train_for_espagne = import_dataset(TRAIN_ESPAGNE_FOLDER, class_espagne_expected_outputs)

class_france_expected_outputs = [0, 1, 0]
dataset_test_for_france = import_dataset(TEST_FRANCE_FOLDER, class_france_expected_outputs)
dataset_train_for_france = import_dataset(TRAIN_FRANCE_FOLDER, class_france_expected_outputs)

class_japon_expected_outputs = [0, 0, 1]
dataset_test_for_japon = import_dataset(TEST_JAPON_FOLDER, class_japon_expected_outputs)
dataset_train_for_japon = import_dataset(TRAIN_JAPON_FOLDER, class_japon_expected_outputs)

final_dataset_test = dataset_test_for_espagne + dataset_test_for_france + dataset_test_for_japon
final_dataset_train = dataset_train_for_espagne + dataset_train_for_france + dataset_train_for_japon

In [7]:
for value in final_dataset_test:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_test.append(the_image)
    y_test.append(the_classe)

for value in final_dataset_train:

    the_image = value["value"].tolist()
    the_classe = value["classe"]

    x_train.append(the_image)
    y_train.append(the_classe)

dataset_inputs = x_train
dataset_outputs = y_train

initiate_dataset_inputs_size = len(dataset_inputs[0])
initiate_dataset_inputs_type = ctypes.c_float * initiate_dataset_inputs_size

classes_initiate_dataset_inputs_size = initiate_dataset_inputs_size * 2

In [8]:
inputs_result = []

for sublist in dataset_inputs:
    for item in sublist:
        inputs_result.append(item)

dataset_inputs = inputs_result

outputs_result = []

for sublist in dataset_outputs:
    for item in sublist:
        outputs_result.append(item)

dataset_outputs = outputs_result

arr = [initiate_dataset_inputs_size, classes_initiate_dataset_inputs_size, 3]
arr_size = len(arr)
arr_type = ctypes.c_int * arr_size

In [9]:
# Convert train/test to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [10]:
#### ------------------------------------------- ####
#### ------------LOAD MODELS---------------------#### 
#### ------------------------------------------- ####

In [11]:
model_pmc_a = tf.keras.models.load_model("../models/keras_models/model_a.h5")

In [12]:
model_pmc_b = tf.keras.models.load_model("../models/keras_models/model_b.h5")

In [13]:
model_pmc_c = tf.keras.models.load_model("../models/keras_models/model_c.h5")

In [14]:
model_pmc_d = tf.keras.models.load_model("../models/keras_models/model_d.h5")

In [15]:
#### ------------------------------------------- ####
#### ------ DEFINE THE TRAINING ALGORITHM ------ #### 
#### ------------------------------------------- ####

In [16]:
OPTIMIZER = tf.keras.optimizers.SGD(learning_rate=1e-3)
LOSS = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='acc'),
]

In [17]:
model_pmc_a.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_b.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_c.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

model_pmc_d.compile(
    optimizer=OPTIMIZER,
    loss=LOSS,
    metrics=METRICS
)

In [18]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (A) ------------- #### 
#### ------------------------------------------- ####

In [37]:
tf.keras.backend.clear_session()

In [45]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_a.h5', save_best_only=True)

In [46]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_a_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_a = model_pmc_a.fit(
    x=x_train, 
    y=y_train, 
    epochs=200, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
        
        
    ]

)

Epoch 1/100
19/19 [==============================] - 1s 64ms/step - loss: 0.1244 - acc: 0.9933 - val_loss: 0.6344 - val_acc: 0.7000
Epoch 2/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1224 - acc: 0.9883 - val_loss: 0.5921 - val_acc: 0.7417
Epoch 3/100
19/19 [==============================] - 0s 8ms/step - loss: 0.1242 - acc: 0.9867 - val_loss: 0.6137 - val_acc: 0.7083
Epoch 4/100
19/19 [==============================] - 0s 9ms/step - loss: 0.1208 - acc: 0.9900 - val_loss: 0.6234 - val_acc: 0.7667
Epoch 5/100
19/19 [==============================] - 0s 8ms/step - loss: 0.1212 - acc: 0.9917 - val_loss: 0.6197 - val_acc: 0.7000
Epoch 6/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1201 - acc: 0.9917 - val_loss: 0.6344 - val_acc: 0.7333
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1226 - acc: 0.9833 - val_loss: 0.6166 - val_acc: 0.7083
Epoch 8/100
19/19 [==============================] - 0s 7ms/step - loss: 0.1209 - 

19/19 [==============================] - 0s 5ms/step - loss: 0.1175 - acc: 0.9883 - val_loss: 0.6512 - val_acc: 0.7167
Epoch 64/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1117 - acc: 0.9900 - val_loss: 0.6299 - val_acc: 0.7417
Epoch 65/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1131 - acc: 0.9883 - val_loss: 0.6521 - val_acc: 0.7167
Epoch 66/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1118 - acc: 0.9917 - val_loss: 0.6164 - val_acc: 0.7083
Epoch 67/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1140 - acc: 0.9917 - val_loss: 0.6411 - val_acc: 0.7000
Epoch 68/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1169 - acc: 0.9867 - val_loss: 0.6193 - val_acc: 0.7750
Epoch 69/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1213 - acc: 0.9817 - val_loss: 0.6518 - val_acc: 0.7167
Epoch 70/100
19/19 [==============================] - 0s 6ms/step - loss: 0.1140 - acc: 0

In [40]:
history_a.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [41]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (B) ------------- #### 
#### ------------------------------------------- ####

In [42]:
tf.keras.backend.clear_session()

In [43]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_b.h5', save_best_only=True)

In [44]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_b_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_b = model_pmc_b.fit(
    x=x_train, 
    y=y_train, 
    epochs=200, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/200
19/19 [==============================] - 1s 59ms/step - loss: 0.1475 - acc: 0.9650 - val_loss: 0.5797 - val_acc: 0.7500
Epoch 2/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1454 - acc: 0.9733 - val_loss: 0.6379 - val_acc: 0.7333
Epoch 3/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1517 - acc: 0.9667 - val_loss: 0.6070 - val_acc: 0.7500
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1609 - acc: 0.9550 - val_loss: 0.6169 - val_acc: 0.7250
Epoch 5/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1542 - acc: 0.9667 - val_loss: 0.5835 - val_acc: 0.7333
Epoch 6/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1483 - acc: 0.9750 - val_loss: 0.5868 - val_acc: 0.7333
Epoch 7/200
19/19 [==============================] - 0s 6ms/step - loss: 0.1646 - acc: 0.9550 - val_loss: 0.6177 - val_acc: 0.7500
Epoch 8/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1595 - 

19/19 [==============================] - 0s 4ms/step - loss: 0.1192 - acc: 0.9833 - val_loss: 0.7223 - val_acc: 0.7417
Epoch 64/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1343 - acc: 0.9717 - val_loss: 0.6035 - val_acc: 0.7417
Epoch 65/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1260 - acc: 0.9750 - val_loss: 0.5990 - val_acc: 0.7417
Epoch 66/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1249 - acc: 0.9833 - val_loss: 0.6797 - val_acc: 0.7250
Epoch 67/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1240 - acc: 0.9783 - val_loss: 0.6299 - val_acc: 0.7417
Epoch 68/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1298 - acc: 0.9767 - val_loss: 0.6562 - val_acc: 0.7583
Epoch 69/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1194 - acc: 0.9783 - val_loss: 0.6306 - val_acc: 0.7500
Epoch 70/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1272 - acc: 0

Epoch 125/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1065 - acc: 0.9833 - val_loss: 0.6479 - val_acc: 0.7333
Epoch 126/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1056 - acc: 0.9850 - val_loss: 0.6719 - val_acc: 0.7417
Epoch 127/200
19/19 [==============================] - 0s 7ms/step - loss: 0.0963 - acc: 0.9917 - val_loss: 0.6791 - val_acc: 0.7750
Epoch 128/200
19/19 [==============================] - 0s 6ms/step - loss: 0.1040 - acc: 0.9867 - val_loss: 0.6408 - val_acc: 0.7333
Epoch 129/200
19/19 [==============================] - 0s 5ms/step - loss: 0.0979 - acc: 0.9900 - val_loss: 0.6977 - val_acc: 0.7417
Epoch 130/200
19/19 [==============================] - 0s 6ms/step - loss: 0.0969 - acc: 0.9867 - val_loss: 0.6497 - val_acc: 0.7417
Epoch 131/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1048 - acc: 0.9850 - val_loss: 0.6512 - val_acc: 0.7333
Epoch 132/200
19/19 [==============================] - 0s 6ms/step - 

19/19 [==============================] - 0s 6ms/step - loss: 0.0871 - acc: 0.9867 - val_loss: 0.6766 - val_acc: 0.7500
Epoch 187/200
19/19 [==============================] - 0s 6ms/step - loss: 0.0858 - acc: 0.9867 - val_loss: 0.6729 - val_acc: 0.7417
Epoch 188/200
19/19 [==============================] - 0s 7ms/step - loss: 0.0929 - acc: 0.9900 - val_loss: 0.6895 - val_acc: 0.7333
Epoch 189/200
19/19 [==============================] - 0s 4ms/step - loss: 0.0892 - acc: 0.9883 - val_loss: 0.7331 - val_acc: 0.7417
Epoch 190/200
19/19 [==============================] - 0s 4ms/step - loss: 0.0840 - acc: 0.9900 - val_loss: 0.6835 - val_acc: 0.7333
Epoch 191/200
19/19 [==============================] - 0s 5ms/step - loss: 0.0875 - acc: 0.9917 - val_loss: 0.7087 - val_acc: 0.7417
Epoch 192/200
19/19 [==============================] - 0s 6ms/step - loss: 0.0812 - acc: 0.9950 - val_loss: 0.7127 - val_acc: 0.7417
Epoch 193/200
19/19 [==============================] - 0s 7ms/step - loss: 0.0801 -

In [26]:
history_b.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [27]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (C) ------------- #### 
#### ------------------------------------------- ####

In [28]:
tf.keras.backend.clear_session()

In [29]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_c.h5', save_best_only=True)

In [30]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_c_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_c = model_pmc_c.fit(
    x=x_train, 
    y=y_train, 
    epochs=200, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/200
19/19 [==============================] - 1s 64ms/step - loss: 0.2354 - acc: 0.8972 - val_loss: 0.4869 - val_acc: 0.7750
Epoch 2/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2262 - acc: 0.9367 - val_loss: 0.5105 - val_acc: 0.7833
Epoch 3/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2221 - acc: 0.9450 - val_loss: 0.5422 - val_acc: 0.7417
Epoch 4/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2360 - acc: 0.9300 - val_loss: 0.4816 - val_acc: 0.7750
Epoch 5/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2264 - acc: 0.9350 - val_loss: 0.5715 - val_acc: 0.7833
Epoch 6/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2148 - acc: 0.9417 - val_loss: 0.5025 - val_acc: 0.7750
Epoch 7/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2299 - acc: 0.9367 - val_loss: 0.4911 - val_acc: 0.8000
Epoch 8/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2194 - 

19/19 [==============================] - 0s 5ms/step - loss: 0.1822 - acc: 0.9533 - val_loss: 0.5820 - val_acc: 0.7583
Epoch 64/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1694 - acc: 0.9633 - val_loss: 0.5752 - val_acc: 0.7833
Epoch 65/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1847 - acc: 0.9600 - val_loss: 0.5274 - val_acc: 0.8000
Epoch 66/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1728 - acc: 0.9617 - val_loss: 0.6138 - val_acc: 0.7667
Epoch 67/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1735 - acc: 0.9567 - val_loss: 0.5189 - val_acc: 0.7917
Epoch 68/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1674 - acc: 0.9633 - val_loss: 0.5103 - val_acc: 0.7667
Epoch 69/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1872 - acc: 0.9517 - val_loss: 0.5000 - val_acc: 0.7750
Epoch 70/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1673 - acc: 0

Epoch 125/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1361 - acc: 0.9817 - val_loss: 0.5355 - val_acc: 0.7833
Epoch 126/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1324 - acc: 0.9750 - val_loss: 0.5073 - val_acc: 0.7833
Epoch 127/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1403 - acc: 0.9767 - val_loss: 0.5369 - val_acc: 0.7750
Epoch 128/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1354 - acc: 0.9750 - val_loss: 0.5257 - val_acc: 0.7667
Epoch 129/200
19/19 [==============================] - 0s 6ms/step - loss: 0.1339 - acc: 0.9717 - val_loss: 0.5480 - val_acc: 0.7500
Epoch 130/200
19/19 [==============================] - 0s 5ms/step - loss: 0.1277 - acc: 0.9800 - val_loss: 0.5362 - val_acc: 0.7750
Epoch 131/200
19/19 [==============================] - 0s 6ms/step - loss: 0.1327 - acc: 0.9817 - val_loss: 0.5442 - val_acc: 0.7833
Epoch 132/200
19/19 [==============================] - 0s 6ms/step - 

19/19 [==============================] - 0s 9ms/step - loss: 0.1062 - acc: 0.9883 - val_loss: 0.7159 - val_acc: 0.7583
Epoch 187/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1081 - acc: 0.9900 - val_loss: 0.5359 - val_acc: 0.7917
Epoch 188/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1073 - acc: 0.9867 - val_loss: 0.5455 - val_acc: 0.7917
Epoch 189/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1042 - acc: 0.9883 - val_loss: 0.5366 - val_acc: 0.7750
Epoch 190/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1094 - acc: 0.9850 - val_loss: 0.6100 - val_acc: 0.7750
Epoch 191/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1047 - acc: 0.9883 - val_loss: 0.5632 - val_acc: 0.7833
Epoch 192/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1026 - acc: 0.9900 - val_loss: 0.5752 - val_acc: 0.8000
Epoch 193/200
19/19 [==============================] - 0s 6ms/step - loss: 0.1024 -

In [31]:
history_c.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [32]:
#### ------------------------------------------- ####
#### ------------- TRAIN MODEL (D) ------------- #### 
#### ------------------------------------------- ####

In [33]:
tf.keras.backend.clear_session()

In [34]:
# Callbacks for saving the checkpoints at every epoch.
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('../models/keras_models/model_d.h5', save_best_only=True)

In [35]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir="../models/keras_models/keras_logs/" + datetime.datetime.now().strftime("model_d_%Y%m%d-%H%M%S"), 
    histogram_freq=1
)

history_d = model_pmc_d.fit(
    x=x_train, 
    y=y_train, 
    epochs=200, 
    validation_data=(x_test, y_test), 
    callbacks=[
        tensorboard_cb,
        checkpoint_cb,
    ]
)

Epoch 1/200
19/19 [==============================] - 2s 75ms/step - loss: 0.2657 - acc: 0.8778 - val_loss: 0.8132 - val_acc: 0.7083
Epoch 2/200
19/19 [==============================] - 0s 6ms/step - loss: 0.2877 - acc: 0.8967 - val_loss: 0.5663 - val_acc: 0.7333
Epoch 3/200
19/19 [==============================] - 0s 8ms/step - loss: 0.2423 - acc: 0.9150 - val_loss: 0.5630 - val_acc: 0.7250
Epoch 4/200
19/19 [==============================] - 0s 8ms/step - loss: 0.2663 - acc: 0.9017 - val_loss: 0.6400 - val_acc: 0.7000
Epoch 5/200
19/19 [==============================] - 0s 7ms/step - loss: 0.2599 - acc: 0.9167 - val_loss: 0.5973 - val_acc: 0.7083
Epoch 6/200
19/19 [==============================] - 0s 8ms/step - loss: 0.2672 - acc: 0.9017 - val_loss: 0.5695 - val_acc: 0.7250
Epoch 7/200
19/19 [==============================] - 0s 7ms/step - loss: 0.2512 - acc: 0.9033 - val_loss: 0.6084 - val_acc: 0.7083
Epoch 8/200
19/19 [==============================] - 0s 7ms/step - loss: 0.2413 - 

19/19 [==============================] - 0s 8ms/step - loss: 0.1930 - acc: 0.9467 - val_loss: 0.5971 - val_acc: 0.7417
Epoch 64/200
19/19 [==============================] - 0s 7ms/step - loss: 0.2129 - acc: 0.9367 - val_loss: 0.7224 - val_acc: 0.7250
Epoch 65/200
19/19 [==============================] - 0s 7ms/step - loss: 0.2471 - acc: 0.9167 - val_loss: 0.5621 - val_acc: 0.7083
Epoch 66/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1792 - acc: 0.9483 - val_loss: 0.5448 - val_acc: 0.7333
Epoch 67/200
19/19 [==============================] - 0s 8ms/step - loss: 0.2018 - acc: 0.9367 - val_loss: 0.5942 - val_acc: 0.7500
Epoch 68/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1962 - acc: 0.9267 - val_loss: 0.5508 - val_acc: 0.7500
Epoch 69/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1842 - acc: 0.9467 - val_loss: 0.6725 - val_acc: 0.7083
Epoch 70/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1955 - acc: 0

Epoch 125/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1414 - acc: 0.9633 - val_loss: 0.7157 - val_acc: 0.7000
Epoch 126/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1213 - acc: 0.9800 - val_loss: 0.6211 - val_acc: 0.7167
Epoch 127/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1526 - acc: 0.9617 - val_loss: 0.6538 - val_acc: 0.7167
Epoch 128/200
19/19 [==============================] - 0s 8ms/step - loss: 0.2275 - acc: 0.9250 - val_loss: 0.8433 - val_acc: 0.7083
Epoch 129/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1784 - acc: 0.9450 - val_loss: 0.6274 - val_acc: 0.7333
Epoch 130/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1240 - acc: 0.9750 - val_loss: 0.5863 - val_acc: 0.7417
Epoch 131/200
19/19 [==============================] - 0s 7ms/step - loss: 0.1313 - acc: 0.9650 - val_loss: 0.7214 - val_acc: 0.7083
Epoch 132/200
19/19 [==============================] - 0s 6ms/step - 

19/19 [==============================] - 0s 8ms/step - loss: 0.0853 - acc: 0.9950 - val_loss: 0.6280 - val_acc: 0.7417
Epoch 187/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1015 - acc: 0.9867 - val_loss: 0.7638 - val_acc: 0.7167
Epoch 188/200
19/19 [==============================] - 0s 8ms/step - loss: 0.1058 - acc: 0.9783 - val_loss: 0.7042 - val_acc: 0.7167
Epoch 189/200
19/19 [==============================] - 0s 11ms/step - loss: 0.0889 - acc: 0.9900 - val_loss: 0.7358 - val_acc: 0.7167
Epoch 190/200
19/19 [==============================] - 0s 6ms/step - loss: 0.0915 - acc: 0.9833 - val_loss: 0.6537 - val_acc: 0.7167
Epoch 191/200
19/19 [==============================] - 0s 7ms/step - loss: 0.0900 - acc: 0.9833 - val_loss: 0.7164 - val_acc: 0.7167
Epoch 192/200
19/19 [==============================] - 0s 10ms/step - loss: 0.1112 - acc: 0.9733 - val_loss: 0.6583 - val_acc: 0.7250
Epoch 193/200
19/19 [==============================] - 0s 8ms/step - loss: 0.0936

In [36]:
history_d.history.keys()

dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])

In [ ]:
#### ------------------------------------------- ####
#### ------------- SHOW CURVES ------------------#### 
#### ------------------------------------------- ####

In [1]:
# Load the TensorBoard notebook extension


In [ ]:
Run tensorboard --logdir "logspath" in terminal